# Student Success Classification

This notebook is intended to be a place where our team experiments, works with, and models the data. Our goal is to classify the status of students between 3 distinct classes:
1. Drop out
2. Graduated
3. Enrolled

There are 36 features to predict this including demographic information, academic information, financial information, and a few economic indicators.

## Importing the data

In [4]:
%%bash
ls data

Impact-mobile-phone-students.csv
student-dropout-academic-success.csv
student-performance.csv


In [5]:
import pandas as pd

# importing data and insuring columns were read in correctly
raw_data = pd.read_csv("data/student-dropout-academic-success.csv", delimiter=";")
raw_data.columns = raw_data.columns.str.strip()
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance                      4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Father's qualification                   

In [6]:
# looking at target column
raw_data.Target.unique()

array(['Dropout', 'Graduate', 'Enrolled'], dtype=object)